In [6]:
import os
import tweepy
import logging

In [3]:
import sys; sys.executable

'/home/avoy/anaconda3/envs/new_cs329s/bin/python'

### Load access credentials to bot

In [4]:
consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')


In [7]:
logger = logging.getLogger()

def create_api():
    consumer_key = os.getenv("CONSUMER_KEY")
    consumer_secret = os.getenv("CONSUMER_SECRET")
    access_token = os.getenv("ACCESS_TOKEN")
    access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error("Error creating API", exc_info=True)
        raise e
    logger.info("API created")
    return api


In [9]:
api = create_api()


In [10]:
lat, lon = 0, 0


Status(_api=<tweepy.api.API object at 0x7fb41c4d6550>, _json={'created_at': 'Fri Mar 12 22:38:34 +0000 2021', 'id': 1370504518895497218, 'id_str': '1370504518895497218', 'text': 'Earthquake at (0, 0)', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1370235376275578885, 'id_str': '1370235376275578885', 'name': 'QuakeFlow', 'screen_name': 'QuakeFlow_bot', 'location': '', 'description': 'QuakeFlow Project', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Fri Mar 12 04:49:20 +0000 2021', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 2, 'lang': None, 'contributors_enabled': Fals

In [11]:
api.update_with_media('quakeflow logo design 2.jpg')

Status(_api=<tweepy.api.API object at 0x7fb41c4d6550>, _json={'created_at': 'Fri Mar 12 22:54:54 +0000 2021', 'id': 1370508628529086464, 'id_str': '1370508628529086464', 'text': 'https://t.co/UFkqK6wQtC', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1370508627413327872, 'id_str': '1370508627413327872', 'indices': [0, 23], 'media_url': 'http://pbs.twimg.com/media/EwUGVdqUYAAX4eL.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EwUGVdqUYAAX4eL.jpg', 'url': 'https://t.co/UFkqK6wQtC', 'display_url': 'pic.twitter.com/UFkqK6wQtC', 'expanded_url': 'https://twitter.com/QuakeFlow_bot/status/1370508628529086464/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 1200, 'h': 62, 'resize': 'fit'}, 'thumb': {'w': 132, 'h': 132, 'resize': 'crop'}, 'large': {'w': 2048, 'h': 106, 'resize': 'fit'}, 'small': {'w': 680, 'h': 35, 'resize': 'fit'}}}]}, 'extended_entities': {'media': [{'id': 1370508627413327872, 'id_str': '13705086274

In [ ]:
import numpy as np

# Lambdas and Constants
normalize = lambda x: (x - np.mean(x) + np.finfo(x.dtype).eps)/(np.std(x)+np.finfo(x.dtype).eps)
timestamp_seconds = lambda x: datetime.fromisoformat(x).timestamp()
wave_dict = defaultdict(list)
pick_dict = defaultdict(list)
event_dict = defaultdict(dict)
event_min_gap = 5

In [ ]:
from kafka import KafkaConsumer

# Connection to Kafka
consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    key_deserializer=lambda x: loads(x.decode('utf-8')),
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

consumer.subscribe(['waveform_raw', 'phasenet_picks', 'gmma_events'])

In [ ]:
# Handle messages from Kafka
for i, message in enumerate(consumer):

    if message.topic == "waveform_raw":
        key = message.key
        timestamp = message.value[0]
        vec = message.value[1]
        wave_dict[key].append(message.value)
        wave_dict[key] = wave_dict[key][-window_number:]
    
    elif message.topic == "phasenet_picks":
        # print("phasenet!")
        key = message.key
        pick = message.value
        pick_dict[key].append(pick)

    elif message.topic == "gmma_events":
        # print("gmma!")
        key = np.round(timestamp_seconds(message.key)/event_min_gap) * event_min_gap
        event = message.value
        # event_list.extend(event)
        # event_dict[key].append(event)
        event_dict[key] = event
    else:
        raise("Topic Error!")

In [ ]:
def get_plot_events(message, t0, tn):
    t0_idx = 0
    t_events = []
    mag_events = []
    loc_events = []
    for k, x in message.items():
        if timestamp_seconds(x["time"]) >= t0:
            # if t0_idx == 0:
            #     t0_idx = i
            if timestamp_seconds(x["time"]) <= tn - 8 :
                t_events.append(timestamp_seconds(x["time"]) - t0)
                mag_events.append(x["magnitude"])
                loc_events.append(x["location"])
            else:
                return t_events, mag_events, loc_events, t0_idx 
    return t_events, mag_events, loc_events, t0_idx 

